In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time
import numpy as np

# Initialize camera and FaceMesh
cam = cv2.VideoCapture(0)
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)
screen_w, screen_h = pyautogui.size()

# Cooldown for left click
blink_cooldown = 1  # 1 second cooldown
last_left_click_time = 0

# Variables for head movement tracking
prev_nose_x, prev_nose_y = None, None
sensitivity = 6  # Increased sensitivity for better accuracy

# Eye blink detection variables
eye_closure_threshold = 0.02  # Threshold for detecting closed eyes
blink_start_time = None  # Track when the blink starts
blink_duration_threshold = 0.3  # Blink must last at least 0.3 seconds

while True:
    ret, frame = cam.read()
    if not ret:
        break  # Exit loop if frame is empty

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    if landmark_points:
        landmarks = landmark_points[0].landmark

        # Cursor Movement using Nose Landmark (Head Movement Control)
        nose = landmarks[1]  # Nose tip landmark
        nose_x, nose_y = int(nose.x * frame_w), int(nose.y * frame_h)

        if prev_nose_x is not None and prev_nose_y is not None:
            dx = (nose_x - prev_nose_x) * sensitivity
            dy = (nose_y - prev_nose_y) * sensitivity
            pyautogui.moveRel(dx, dy)

        prev_nose_x, prev_nose_y = nose_x, nose_y

        # Eye Blink Detection (Both Eyes for Left Click)
        left_eye_top = landmarks[159].y
        left_eye_bottom = landmarks[145].y
        right_eye_top = landmarks[386].y
        right_eye_bottom = landmarks[374].y

        left_eye_ratio = abs(left_eye_top - left_eye_bottom)
        right_eye_ratio = abs(right_eye_top - right_eye_bottom)

        eyes_closed = left_eye_ratio < eye_closure_threshold and right_eye_ratio < eye_closure_threshold

        if eyes_closed:
            if blink_start_time is None:  # Blink just started
                blink_start_time = time.time()
        else:
            if blink_start_time is not None:
                blink_duration = time.time() - blink_start_time
                if blink_duration >= blink_duration_threshold:
                    if time.time() - last_left_click_time > blink_cooldown:
                        pyautogui.click()  # Left Click
                        last_left_click_time = time.time()
                blink_start_time = None  # Reset blink timer

    cv2.imshow('Eye Controlled Mouse', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cam.release()
cv2.destroyAllWindows()
 